<a href="https://colab.research.google.com/github/EsrinErdem/Movie-Recommandation-System/blob/main/CaeaFlix_Recommandation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAEAFLIX - RECOMMANDATION SYSTEM 

In [ ]:
#We import usefull libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df=pd.read_csv('df_finish_v4')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df.shape

(14842, 24)

In [ ]:
df.columns

Index(['id_tmdb', 'tconst', 'movies_keywords', 'companies', 'movies_websites',
       'movies_popularity', 'nconst', 'primaryTitle', 'genres',
       'movies_poster', 'movies_budget', 'movies_overview', 'movies_revenue',
       'iso_global', 'actors', 'actress', 'producer', 'director', 'writer',
       'startYear', 'runtimeMinutes', 'averageRating', 'numVotes',
       'diffusion'],
      dtype='object')

In [ ]:
#We select the features to use for the recommandation system (only string)
features = ['movies_keywords', 'primaryTitle','genres', 'director','actors','actress','companies','movies_overview']

#We clean them
for feature in features:
    df[feature] = df[feature].str.replace("[","")
    df[feature] = df[feature].str.replace("]","")
    df[feature] = df[feature].fillna('')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys


In [ ]:
df[['movies_keywords', 'primaryTitle','genres', 'director','actors','actress','companies']]

,movies_keywords,primaryTitle,genres,director,actors,actress,companies
0,"'lover (female)', ""love of one's life"", 'time ...",Kate & Leopold,"'Comedy', 'Fantasy', 'Romance'",'James Mangold',"'Hugh Jackman', 'Liev Schreiber', 'Breckin Meyer'",'Meg Ryan',"'Konrad Pictures', 'Miramax'"
1,"'1970s', 'comeback', 'satire', 'film in film',...",The Other Side of the Wind,'Drama','Orson Welles',"'John Huston', 'Peter Bogdanovich'","'Susan Strasberg', 'Oja Kodar'","'SACI', ""Les Films de l'Astrophore"""
2,,"Libera, My Love",'Drama','Mauro Bolognini',"'Adolfo Celi', 'Philippe Leroy', 'Bruno Cirino'",'Claudia Cardinale','Roberto Loyola Cinematografica'
3,"'fairy tale', 'anime'",The Little Mermaid,"'Adventure', 'Drama', 'Animation'","'Tomoharu Katsumata', 'Tim Reid'","'Tarô Shigaki', 'Hideki Shibata'","'Mariko Miyagi', 'Fumie Kashiyama'",'Toei Animation'
4,"'biography', 'film directing'",Kenji Mizoguchi: The Life of a Film Director,"'Biography', 'Documentary'",'Kaneto Shindô','unknown','unknown','Kindai Eiga Kyokai'
...,...,...,...,...,...,...,...
14837,,Herself,'Drama','Phyllida Lloyd','Ian Lloyd Anderson',"'Clare Dunne', 'Molly McCann', ""Ruby Rose O'Hara""","'Element Pictures', 'BBC Films', 'Merman', 'Am..."
14838,,Jessie,"'Thriller', 'Horror'",'Aswani Kumar V.',"'Atul Kulkarni', 'Kabir Duhan Singh'","'Ashima Narwal', 'Sritha Chandana'",
14839,"'gypsy', 'world war i', 'flashback', 'gore', '...",The Cursed,"'Fantasy', 'Horror', 'Mystery'",'Sean Ellis',"'Alistair Petrie', 'Boyd Holbrook'","'Kelly Reilly', 'Roxane Duran'","'Piste Rouge', 'LD Entertainment', 'Climax Stu..."
14840,'espionage',Le lion,'Comedy','Ludovic Colbeau-Justin',"'Dany Boon', 'Samuel Jouy', 'Philippe Katerine'",'Anne Serra',"'TF1 Studio', 'Monkey Pack Films', 'Pathé', 'T..."


In [ ]:
#We combine each features of each rows together
def combined_features(row):
    return row['movies_keywords']+" "+row['genres']+" "+row['companies']+" "+row['actors']+" "+row['actress']+" "+row['director']+" "+row['primaryTitle']
df["combined_features"] = df.apply(combined_features, axis =1)

In [ ]:
#We use NLP to count each words presence and create binary lists 
cv = CountVectorizer()

#We train and scale our features 
count_matrix = cv.fit_transform(df["combined_features"])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
df[df.primaryTitle.str.contains('Boss')]

,id_tmdb,tconst,movies_keywords,companies,movies_websites,movies_popularity,nconst,primaryTitle,genres,movies_poster,...,actress,producer,director,writer,startYear,runtimeMinutes,averageRating,numVotes,diffusion,combined_features
6825,5206.0,tt0469754,"'office', 'it-expert'","'Zentropa Entertainments', 'Memfis Film', 'Slo...",http://www.direktorenfordethele.dk/,10.263,"['nm0001885', 'nm0934668', 'nm0296144', 'nm030...",The Boss of It All,'Comedy',https://www.themoviedb.org/t/p/w600_and_h900_b...,...,'unknown',"['Vibeke Windeløv', 'Meta Louise Foldager Søre...",'Lars von Trier',['unknown'],2006,99.0,6.6,11581,23,"'office', 'it-expert' 'Comedy' 'Zentropa Enter..."
9578,51540.0,tt1499658,"'bad boss', 'employee', 'death of father', 'st...","'New Line Cinema', 'Rat Entertainment'",http://horriblebossesmovie.warnerbros.com/inde...,30.626,"['nm0000867', 'nm0711840', 'nm0197855', 'nm000...",Horrible Bosses,"'Comedy', 'Crime'",https://www.themoviedb.org/t/p/w600_and_h900_b...,...,'unknown',"['Brett Ratner', 'Jay Stern']",'Seth Gordon',"['John Francis Daley', 'Michael Markowitz', 'J...",2011,98.0,6.9,441017,42,"'bad boss', 'employee', 'death of father', 'st..."
10740,227159.0,tt2170439,"'prison', 'kidnapping', 'sex addiction', 'sequ...","'New Line Cinema', 'BenderSpink', 'RatPac Ente...",NaN,31.619,"['nm0000867', 'nm0000098', 'nm0197855', 'nm007...",Horrible Bosses 2,"'Comedy', 'Crime'",https://www.themoviedb.org/t/p/w600_and_h900_b...,...,'Jennifer Aniston',['Chris Bender'],'Sean Anders',"['John Francis Daley', 'Michael Markowitz', 'J...",2014,108.0,6.3,169089,43,"'prison', 'kidnapping', 'sex addiction', 'sequ..."
11246,151036.0,tt2577622,"'cheating', 'emigration', 'visa', 'train', 'fa...",,NaN,1.596,"['nm0756533', 'nm0663967', 'nm0226770', 'nm195...",My Boss,'Comedy',https://www.themoviedb.org/t/p/w600_and_h900_b...,...,"'Seetha', 'Mamta Mohandas'",['East Coast Vijayan'],'Jeethu Joseph',['unknown'],2012,160.0,6.7,1473,7,"'cheating', 'emigration', 'visa', 'train', 'fa..."
11316,323676.0,tt2702724,"'business woman', 'ex-con', 'duringcreditsstin...","'Universal Pictures', 'On the Day', 'Gary Sanc...",http://www.thebossfilm.com/,14.436,"['nm0376260', 'nm0002071', 'nm0565250', 'nm022...",The Boss,'Comedy',https://www.themoviedb.org/t/p/w600_and_h900_b...,...,"'Melissa McCarthy', 'Kristen Bell', 'Ella Ande...","['Chris Henchy', 'Will Ferrell', 'Adam McKay']",'Ben Falcone',['Steve Mallory'],2016,99.0,5.4,45680,36,"'business woman', 'ex-con', 'duringcreditsstin..."
12097,295693.0,tt3874544,"'baby', 'family relationships', 'sibling rival...",'DreamWorks Animation',http://www.dreamworks.com/thebossbaby/,40.770,"['nm0000285', 'nm0000114', 'nm0569891', 'nm000...",The Boss Baby,"'Adventure', 'Comedy', 'Animation'",https://www.themoviedb.org/t/p/w600_and_h900_b...,...,'Lisa Kudrow',['Ramsey Naito'],'Tom McGrath',"['Michael McCullers', 'Marla Frazee']",2017,97.0,6.3,122204,49,"'baby', 'family relationships', 'sibling rival..."
13623,459151.0,tt6932874,"'baby', 'boss', 'sequel', 'sibling', 'family',...","'DreamWorks Animation', 'Universal Pictures'",https://www.dreamworks.com/movies/the-boss-baby-2,517.176,"['nm0000285', 'nm0000285', 'nm0000285', 'nm056...",The Boss Baby 2: Family Business,"'Adventure', 'Comedy', 'Animation'",https://www.themoviedb.org/t/p/w600_and_h900_b...,...,"'Amy Sedaris', 'Amy Sedaris', 'Amy Sedaris', '...","['Jeff Hermann', 'Jeff Hermann', 'Jeff Hermann']","'Tom McGrath', 'Tom McGrath', 'Tom McGrath'","['Michael McCullers', 'Michael McCullers', 'Mi...",2021,107.0,5.9,15992,103,"'baby', 'boss', 'sequel', 'sibling', 'family',..."
13935,513310.0,tt7638348,'time loop',"'WarParty Films', 'Highland Film Group', 'Inge...",https://www.hulu.com/movie/boss-level-622e81d6...,40.273,"['nm0000154', 'nm0000154', 'nm0000154', 'nm000...",Boss Level,"'Mystery', 'Thriller', 'Sci-Fi'",https://www.themoviedb.org/t/p/w600_and_h900_b...,...,"'Michelle Yeoh', 'Michelle Yeoh', 'Michelle Ye...","['Randall Emmett', 'Randall Emmett', 'Randall ...","'Joe

In [ ]:
#We create the recommandation system function
movie_user_likes = "Titanic"

#We find the index with the title entered by the consumer
def get_index_from_title(primaryTitle):
    return df[df.primaryTitle == primaryTitle].index.values[0]
movie_index = get_index_from_title(movie_user_likes)

#We use the index to create a list of similar movies with cosine_sim function which will compare the binary lists
cosine_sim = cosine_similarity(count_matrix)
similar_movies = list(enumerate(cosine_sim[movie_index]))

#We sort by the best scores
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

#We display the results
def get_title_from_index(index):
    return df[df.index == index][["primaryTitle",'movies_overview','movies_poster','movies_websites','actors','actress','director','genres','averageRating','startYear']].values[0]
i=0

#We don't want to display the nearest neigbour which is actually the movie the consumer typed
for movie in sorted_similar_movies[0:]:
  if movie[0] != movie_index:
      print(get_title_from_index(movie[0]))
      i=i+1
      #We fix the number of recommandations we want to display
      print()
      if i>3:
          break

['Endings, Beginnings'
 'An artist dumps her longtime boyfriend, but her attempt to take a break from dating ends when she quickly finds herself in two passionate romances.'
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/5yfJDjTp6n8QXN0M9aGCYpqahXZ.jpg'
 'http://www.samuelgoldwynfilms.com/endings-beginnings/'
 "'Matthew Gray Gubler', 'Sebastian Stan', 'Jamie Dornan'"
 "'Shailene Woodley'" "'Drake Doremus'" "'Drama'" 5.6 2019]

['White Palace'
 "Max Baron is a Jewish advertising executive in his 20s who's still getting over the death of his wife. Nora Baker is a 40-something diner waitress who enjoys the wilder side of life. Mismatched or not, their attraction is instant and smoldering. With time, however, their class and age differences become an obstacle in their relationship, especially since Max can't keep Nora a secret from his Jewish friends and upper-crust associates forever."
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/8H63Gjhfa5f2zkXgEuFA2ThdDJ6.jpg'
 nan "'Jam

Next steps :
* Recommandations based on director only
* Recommandations based on companies only
* Recommandations based on actors/actress only
* Split in two df : movies / anime or three df / vintage ?
* Tenter du NLP sur la colonne overview et primaryTitle

# Recommandations based on director only

In [ ]:
#We combine each features of each rows together
def combined_features(row):
    return row['director']
df["combined_features_director"] = df.apply(combined_features, axis =1)

In [ ]:
#We use NLP to count each words presence and create binary lists 
cv = CountVectorizer()

#We train and scale our features 
count_matrix1 = cv.fit_transform(df["combined_features_director"])
print("Count Matrix:", count_matrix1.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
#We create the recommandation system function
movie_user_likes = "Titanic"

#We find the index with the title entered by the consumer
def get_index_from_title(primaryTitle):
    return df[df.primaryTitle == primaryTitle].index.values[0]
movie_index = get_index_from_title(movie_user_likes)

#We use the index to create a list of similar movies with cosine_sim function which will compare the binary lists
cosine_sim = cosine_similarity(count_matrix1)
similar_movies = list(enumerate(cosine_sim[movie_index]))

#We sort by the best scores
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

#We display the results
def get_title_from_index(index):
    return df[df.index == index][["primaryTitle",'movies_overview','movies_poster','movies_websites','actors','actress','director','genres','averageRating','startYear']].values[0]
i=0

#We don't want to display the nearest neigbour which is actually the movie the consumer typed
for movie in sorted_similar_movies[0:]:
    if movie[0] != movie_index:
        print(get_title_from_index(movie[0]))
        i=i+1
        #We fix the number of recommandations we want to display
        print()
        if i>3:
            break

['Aliens'
 "When Ripley's lifepod is found by a salvage crew over 50 years later, she finds that terra-formers are on the very planet they found the alien species. When the company sends a family of colonists out to investigate her story—all contact is lost with the planet and colonists. They enlist Ripley and the colonial marines to return and search for answers."
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/r1x5JGpyqZU8PYhbs4UcrO1Xb6x.jpg'
 'https://www.20thcenturystudios.com/movies/aliens'
 "'Michael Biehn', 'Michael Biehn', 'Michael Biehn', 'Michael Biehn', 'Michael Biehn', 'Michael Biehn', 'Michael Biehn', 'Paul Reiser', 'Paul Reiser', 'Paul Reiser', 'Paul Reiser', 'Paul Reiser', 'Paul Reiser', 'Paul Reiser'"
 "'Sigourney Weaver', 'Sigourney Weaver', 'Sigourney Weaver', 'Sigourney Weaver', 'Sigourney Weaver', 'Sigourney Weaver', 'Sigourney Weaver', 'Carrie Henn', 'Carrie Henn', 'Carrie Henn', 'Carrie Henn', 'Carrie Henn', 'Carrie Henn', 'Carrie Henn'"
 "'James Cameron', '

In [ ]:
df.columns

Index(['id_tmdb', 'tconst', 'movies_keywords', 'companies', 'movies_websites',
       'movies_popularity', 'nconst', 'primaryTitle', 'genres',
       'movies_poster', 'movies_budget', 'movies_overview', 'movies_revenue',
       'iso_global', 'actors', 'actress', 'producer', 'director', 'writer',
       'startYear', 'runtimeMinutes', 'averageRating', 'numVotes', 'diffusion',
       'combined_features', 'combined_features_director'],
      dtype='object')

# Recommandations based on companies only

In [ ]:
#We combine each features of each rows together
def combined_features(row):
    return row['companies']
df["combined_features_companies"] = df.apply(combined_features, axis =1)

In [ ]:
#We use NLP to count each words presence and create binary lists 
cv = CountVectorizer()

#We train and scale our features 
count_matrix2 = cv.fit_transform(df["combined_features_companies"])
print("Count Matrix:", count_matrix2.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
#We create the recommandation system function
movie_user_likes = "Titanic"

#We find the index with the title entered by the consumer
def get_index_from_title(primaryTitle):
    return df[df.primaryTitle == primaryTitle].index.values[0]
movie_index = get_index_from_title(movie_user_likes)

#We use the index to create a list of similar movies with cosine_sim function which will compare the binary lists
cosine_sim = cosine_similarity(count_matrix2)
similar_movies = list(enumerate(cosine_sim[movie_index]))

#We sort by the best scores
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

#We display the results
def get_title_from_index(index):
    return df[df.index == index][["primaryTitle",'movies_overview','companies','movies_poster','movies_websites','actors','actress','director','genres','averageRating','startYear']].values[0]
i=0

#We don't want to display the nearest neigbour which is actually the movie the consumer typed
for movie in sorted_similar_movies[0:]:
    if movie[0] != movie_index:
        print(get_title_from_index(movie[0]))
        i=i+1
        #We fix the number of recommandations we want to display
        print()
        if i>3:
            break

['True Lies'
 'A fearless, globe-trotting, terrorist-battling secret agent has his life turned upside down when he discovers his wife might be having an affair with a used car salesman while terrorists smuggle nuclear war heads into the United States.'
 "'Lightstorm Entertainment', '20th Century Fox'"
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/pweFTnzzTfGK68woSVkiTgjLzWm.jpg'
 nan "'Arnold Schwarzenegger', 'Bill Paxton', 'Tom Arnold'"
 "'Jamie Lee Curtis'" "'James Cameron'" " 'Thriller', 'Action',  'Comedy'"
 7.3 1994]

['Strange Days'
 'Set in the year 1999 during the last days of the old millennium, the movie tells the story of Lenny Nero, an ex-cop who now deals with data-discs containing recorded memories and emotions. One day he receives a disc which contains the memories of a murderer killing a prostitute. Lenny investigates and is pulled deeper and deeper in a whirl of blackmail, murder and rape. Will he survive and solve the case?'
 "'Lightstorm Entertainment', '20th

# Recommandations based on actors/actress only

In [ ]:
def combined_features(row):
        return row['actors']+" "+row['actress']
df["combined_features_actors_actress"] = df.apply(combined_features, axis =1)
cv = CountVectorizer()
#We train and scale our features 
count_matrix3 = cv.fit_transform(df["combined_features_actors_actress"])
print("Count Matrix:", count_matrix3.toarray())
#We create the recommandation system function
movie_user_likes = "Titanic"

#We find the index with the title entered by the consumer
def get_index_from_title(primaryTitle):
    return df[df.primaryTitle == primaryTitle].index.values[0]
movie_index = get_index_from_title(movie_user_likes)

#We use the index to create a list of similar movies with cosine_sim function which will compare the binary lists
cosine_sim = cosine_similarity(count_matrix3)
similar_movies = list(enumerate(cosine_sim[movie_index]))

#We sort by the best scores
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

#We display the results
def get_title_from_index(index):
    return df[df.index == index][["primaryTitle",'movies_overview','companies','movies_poster','movies_websites','actors','actress','director','genres','averageRating','startYear']].values[0]
i=0

#We don't want to display the nearest neigbour which is actually the movie the consumer typed
for movie in sorted_similar_movies[0:]:
    if movie[0] != movie_index:
        print(get_title_from_index(movie[0]))
        i=i+1
        #We fix the number of recommandations we want to display
        print()
        if i>3:
            break

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['Revolutionary Road'
 'A young couple living in a Connecticut suburb during the mid-1950s struggle to come to terms with their personal problems while trying to raise their two children. Based on a novel by Richard Yates.'
 "'Scott Rudin Productions', 'BBC Films', 'Neal Street Productions', 'Evamere Entertainment', 'DreamWorks Pictures', 'Goldcrest'"
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/1jYKJEGcgbkEjlKuY94Dv7BVe1P.jpg'
 nan "'Leonardo DiCaprio', 'Christopher Fitzgerald', 'Jonathan Roumie'"
 "'Kate Winslet'" "'Sam Mendes'" "'Drama',  'Romance'" 7.3 2008]

['The Aviator'
 'A biopic depicting the life of filmmaker and aviation pioneer Howard Hughes from 1927 to 1947, during which time he became a successful film producer and an aviation magnate, while simultaneously growing more unstable due to severe obsessive-compulsive disorder.'
 '\'Mi

# Split in two df : movies / anime or three df / vintage ?

# Tenter du NLP sur la colonne overview et primaryTitle

In [ ]:
df.columns

Index(['id_tmdb', 'tconst', 'movies_keywords', 'companies', 'movies_websites',
       'movies_popularity', 'nconst', 'primaryTitle', 'genres',
       'movies_poster', 'movies_budget', 'movies_overview', 'movies_revenue',
       'iso_global', 'actors', 'actress', 'producer', 'director', 'writer',
       'startYear', 'runtimeMinutes', 'averageRating', 'numVotes', 'diffusion',
       'combined_features', 'combined_features_director',
       'combined_features_companies', 'combined_features_actors_actress'],
      dtype='object')

In [ ]:
def combined_features(row):
      return row['movies_overview']+" "+row['genres']+" "+row['companies']+" "+row['actors']+" "+row['actress']+" "+row['director']+" "+row['primaryTitle']
df["combined_features_overview"] = df.apply(combined_features, axis =1)
cv = CountVectorizer()
#We train and scale our features 
count_matrix4 = cv.fit_transform(df["combined_features_actors_actress"])
print("Count Matrix:", count_matrix4.toarray())
#We create the recommandation system function
movie_user_likes = "Titanic"

#We find the index with the title entered by the consumer
def get_index_from_title(primaryTitle):
    return df[df.primaryTitle == primaryTitle].index.values[0]
movie_index = get_index_from_title(movie_user_likes)

#We use the index to create a list of similar movies with cosine_sim function which will compare the binary lists
cosine_sim = cosine_similarity(count_matrix4)
similar_movies = list(enumerate(cosine_sim[movie_index]))

#We sort by the best scores
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

#We display the results
def get_title_from_index(index):
    return df[df.index == index][["primaryTitle",'movies_overview','companies','movies_poster','movies_websites','actors','actress','director','genres','averageRating','startYear']].values[0]
i=0

#We don't want to display the nearest neigbour which is actually the movie the consumer typed
for movie in sorted_similar_movies[0:]:
    if movie[0] != movie_index:
        print(get_title_from_index(movie[0]))
        i=i+1
        #We fix the number of recommandations we want to display
        print()
        if i>3:
            break

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['Revolutionary Road'
 'A young couple living in a Connecticut suburb during the mid-1950s struggle to come to terms with their personal problems while trying to raise their two children. Based on a novel by Richard Yates.'
 "'Scott Rudin Productions', 'BBC Films', 'Neal Street Productions', 'Evamere Entertainment', 'DreamWorks Pictures', 'Goldcrest'"
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/1jYKJEGcgbkEjlKuY94Dv7BVe1P.jpg'
 nan "'Leonardo DiCaprio', 'Christopher Fitzgerald', 'Jonathan Roumie'"
 "'Kate Winslet'" "'Sam Mendes'" "'Drama',  'Romance'" 7.3 2008]

['The Aviator'
 'A biopic depicting the life of filmmaker and aviation pioneer Howard Hughes from 1927 to 1947, during which time he became a successful film producer and an aviation magnate, while simultaneously growing more unstable due to severe obsessive-compulsive disorder.'
 '\'Mi

# overview avec nlp nettoiyage

In [ ]:
# !pip install -U pip setuptools wheel
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [ ]:
import nltk
nltk.download('popular')
import spacy
nlp = spacy.load('en_core_web_sm')
tokenizer=nltk.RegexpTokenizer(r"\w+")
def clean(sentence="You are better when I am well."):
  sent_tokens = nlp(sentence)  
  lema_str = ' '.join(word.lemma_ for word in sent_tokens)
  lema_str=lema_str.lower()
  txt_puntuation=tokenizer.tokenize(lema_str)
  tokens_clean = []
  for words in txt_puntuation:
    if words not in nltk.corpus.stopwords.words("english"):
      tokens_clean.append(words)
  tokens_str = ' '.join(word for word in tokens_clean)
  return tokens_str
clean()

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

'pron well pron well'

In [ ]:
df['clean_overview']=df['movies_overview'].apply(lambda x: clean(x))
df['clean_overview']

0        pron scientist ex boyfriend discover portal tr...
1        surround fan skeptic grizzle director j j jake...
2        director mauro bolognini insert story fascist ...
3        story beautiful mermaid give kingdom pron belo...
4        kenji mizoguchi life film director aru eiga ka...
                               ...                        
14837    struggle provide pron daughter safe happy home...
14838    set abandon house film follow team professiona...
14839    late 19th century brutal land baron slaughter ...
14840    psychiatric hospital patient pretend crazy cha...
14841    french basque country year 1609 man small fish...
Name: clean_overview, Length: 14842, dtype: object

In [ ]:
features = ['clean_overview']

#We clean them
for feature in features:
    df[feature] = df[feature].str.replace("[","")
    df[feature] = df[feature].str.replace("]","")
    df[feature] = df[feature].fillna('')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
def combined_features(row):
      return row['clean_overview']+" "+row['genres']+" "+row['companies']+" "+row['actors']+" "+row['actress']+" "+row['director']+" "+row['primaryTitle']
df["combined_features_overview_cleaned"] = df.apply(combined_features, axis =1)
cv = CountVectorizer()
#We train and scale our features 
count_matrix5 = cv.fit_transform(df["combined_features_actors_actress"])
print("Count Matrix:", count_matrix5.toarray())
#We create the recommandation system function
movie_user_likes = "Titanic"

#We find the index with the title entered by the consumer
def get_index_from_title(primaryTitle):
    return df[df.primaryTitle == primaryTitle].index.values[0]
movie_index = get_index_from_title(movie_user_likes)

#We use the index to create a list of similar movies with cosine_sim function which will compare the binary lists
cosine_sim = cosine_similarity(count_matrix5)
similar_movies = list(enumerate(cosine_sim[movie_index]))

#We sort by the best scores
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

#We display the results
def get_title_from_index(index):
    return df[df.index == index][["primaryTitle",'movies_overview','companies','movies_poster','movies_websites','actors','actress','director','genres','averageRating','startYear']].values[0]
i=0

#We don't want to display the nearest neigbour which is actually the movie the consumer typed
for movie in sorted_similar_movies[0:]:
    if movie[0] != movie_index:
        print(get_title_from_index(movie[0]))
        i=i+1
        #We fix the number of recommandations we want to display
        print()
        if i>3:
            break

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['Revolutionary Road'
 'A young couple living in a Connecticut suburb during the mid-1950s struggle to come to terms with their personal problems while trying to raise their two children. Based on a novel by Richard Yates.'
 "'Scott Rudin Productions', 'BBC Films', 'Neal Street Productions', 'Evamere Entertainment', 'DreamWorks Pictures', 'Goldcrest'"
 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/1jYKJEGcgbkEjlKuY94Dv7BVe1P.jpg'
 nan "'Leonardo DiCaprio', 'Christopher Fitzgerald', 'Jonathan Roumie'"
 "'Kate Winslet'" "'Sam Mendes'" "'Drama',  'Romance'" 7.3 2008]

['The Aviator'
 'A biopic depicting the life of filmmaker and aviation pioneer Howard Hughes from 1927 to 1947, during which time he became a successful film producer and an aviation magnate, while simultaneously growing more unstable due to severe obsessive-compulsive disorder.'
 '\'Mi

In [ ]:
# Les mêmes résultats ont été obtenus même 
# si le nettoyage a été fait. Je pense qu'il vaudra mieux utiliser 
# key words à la place en termes de recommandation.